In [ ]:
#!/usr/bin/env python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import scipy.stats as stats
import seaborn as sns

%matplotlib inline

In [ ]:
# sleep 데이터 [Scheffe (1959)]
data = {"no.": range(1, 21),
        "extra": [0.7, -1.6, -0.2, -1.2, -0.1,
                  3.4,  3.7,  0.8,  0.0,  2.0,
                  1.9,  0.8,  1.1,  0.1, -0.1,
                  4.4,  5.5,  1.6,  4.6,  3.4],
        "group": [(x // 10) + 1 for x in range(20)],
        "ID": list(range(1, 11)) * 2}
       
sleep = pd.DataFrame(data=data,
                     columns=["no.", "extra", "group", "ID"])
sleep.set_index("no.")
sleep

In [ ]:
# 약제 1을 복용했을 때 수면시간의 증가 (단위는 시간이다)
y = sleep.loc[sleep["group"]==1]["extra"]
print(y)

y.describe()

In [ ]:
# histogram parameters
# note that the bins are inclusive of their lower bounds,
# and exclusive of their upper bounds
bin_width = 1
y_min = np.rint(y.min())
y_max = np.rint(y.max()) + 1

# set space between subplots
fig = plt.figure(figsize=(10, 10))
fig.subplots_adjust(hspace=.5, wspace=.5)

# histogram
ax1 = fig.add_subplot(221)
ax1.set_title("Histogram of y")
ax1.set_xlabel("y")
ax1.set_ylabel("Frequency")
sns.distplot(y, ax=ax1, bins=np.arange(y_min, y_max, bin_width),
             kde=False)
ax1.set_xticks(np.arange(y_min, y_max))
ax1.set_xlim(y_min, y_max-1)

# boxplot
ax2 = fig.add_subplot(222)
ax2.set_title("Boxplot of y")
ax2.set_xlabel("group")
ax2.set_ylabel("extra")
sns.boxplot(data=y, ax=ax2)
ax2.set_yticks(np.arange(y_min, y_max))

# normal q-q plot
ax3 = fig.add_subplot(223)
#plt.title("Normal Q-Q plot")
z = (y-np.mean(y)) / np.std(y)
res = stats.probplot(z, dist="norm", plot=plt)

# probability density 
ax4 = fig.add_subplot(224)
ax4.set_title("Histogram of y")
ax4.set_xlabel("y")
ax4.set_ylabel("Density")
sns.distplot(y, ax=ax4, bins=np.arange(y_min, y_max),
             hist=True)
ax4.set_xticks(np.arange(y_min, y_max))
ax4.set_xlim(y_min, y_max-1)

plt.show()

In [ ]:
# perform t-test on y
t, pvalue = stats.ttest_1samp(y, .0)
# calculate confidence interval
confidence_level = 0.95
low, high = stats.t.interval(confidence_level, len(y)-1, 
                              loc=np.mean(y), scale=stats.sem(y))

print("data: y")
print("t = % .4f, p-value = %.4f" % (t, pvalue))
print("alternative hypothesis: true mean is not equal to 0")
print("%d percent confidence interval:\n%.7f\t%.7f" % (confidence_level*100, low, high))

In [ ]:
# 개개인의 수면시간증가값 모형
# 평균이 0이고, 표준편차가 1.8(시간)인 종 모양의 분포(bell shaped distribution)
# N(0, 1.8^2)
mu = 0
variance = 1.8**2
sigma = 1.8
np.random.seed(1708)

x = np.linspace(mu - 3*variance, mu + 3*variance, 100)

fig, ax = plt.subplots()
ax.plot(x, mlab.normpdf(x, mu, sigma))
ax.set_xlim(-4, 4)

plt.show()

In [ ]:
# 10,000개의 평행우주의 표본, (각 표본은 10개의 관측치를 포함한다)
# 그리고 각 표본의 평균값, 표본표준편차, 그리고 t-통계량 값을  계산할 수 있다
B = 10000
n = 10

mu = 0
varaince = 1.789**2
sigma = 1.789
np.random.seed(1708)

xbars_star = np.zeros(B, dtype=float)
sds_star = np.zeros(B, dtype=float)
ts_star = np.zeros(B, dtype=float)
for b in range(B):
    y_star = np.random.normal(mu, sigma, n)
    m = y_star.mean()
    s = y_star.std()
    xbars_star[b] = m
    sds_star[b] = s
    ts_star[b] =  (m / (s / np.sqrt(n)))

In [ ]:
# set space between subplots
plt.figure(figsize=(10, 10))
plt.subplots_adjust(hspace=.5, wspace=.5)

# histogram
plt.subplot(221)
plt.title("Histogram of xbars_star")
plt.xlabel("xbars_star")
plt.ylabel("Frequency")
sns.distplot(xbars_star.T, kde=False)
plt.axvline(x=0.75)

plt.subplot(222)
plt.title("Histogram of sds_star")
plt.xlabel("sds_star")
plt.ylabel("Frequency")
sns.distplot(sds_star, kde=False)
plt.axvline(x=1.789)

plt.subplot(223)
plt.title("Histogram of ts_star")
plt.xlabel("ts_star")
plt.ylabel("Frequency")
sns.distplot(ts_star, kde=False)
plt.axvline(x=1.3257)

# normal q-q plot
plt.subplot(224)
z = (ts_star-np.mean(ts_star)) / np.std(ts_star)
stats.probplot(z, dist="norm", plot=plt)

plt.show()

# Calculate p-value manually
pvalue = np.argwhere(ts_star > 1.3257).size / B
print(pvalue)

In [ ]:
# 스튜던트 t 분포
# 다양한 자유도 값에 따른 t 밀도함수
INF = 999
nlist = np.array([1, 2, 5, INF])
x = np.arange(-5.0, 5.05, 0.05)

fig, ax = plt.subplots()
ax.set_xlim(-5, 5)
ax.set_ylim(0, 0.45)

for df in nlist:
    ax.plot(x, stats.t.pdf(x, df), label="df={}".format(df))

ax.legend()
plt.show()

In [ ]:
# 8. 신뢰구간의 의미

n = 10
mu = 1
sigma = 1.8
confidence_level = 0.95

# calculate confidence interval on different set of values
# from the same probability distribution
np.random.seed(1708)
y_star = np.random.normal(mu, sigma, n)
print(stats.t.interval(confidence_level, len(y_star)-1, 
                       loc=np.mean(y_star), scale=stats.sem(y_star)))

y_star = np.random.normal(mu, sigma, n)
print(stats.t.interval(confidence_level, len(y_star)-1, 
                       loc=np.mean(y_star), scale=stats.sem(y_star)))

y_star = np.random.normal(mu, sigma, n)
print(stats.t.interval(confidence_level, len(y_star)-1, 
                       loc=np.mean(y_star), scale=stats.sem(y_star)))

In [ ]:
# run simulation 
B = 100
n = 10

conf_intervals = pd.DataFrame(
    data={
        "b": np.zeros(B, dtype=int),
        "lower": np.zeros(B, dtype=float),
        "xbar": np.zeros(B, dtype=float),
        "upper": np.zeros(B, dtype=float)
    }
)

true_mu = 1.0
sigma = 1.8
confidence_level = 0.95
np.random.seed(1708)

plt.figure(figsize=(10, 10))

for b in range(B):
    y_star = np.random.normal(true_mu, sigma, n)
    lower, upper = stats.t.interval(confidence_level, len(y_star)-1, 
                                    loc=np.mean(y_star), scale=stats.sem(y_star))
    conf_intervals.loc[b, "b"] = b
    conf_intervals.loc[b, "lower"] = lower
    conf_intervals.loc[b, "xbar"] = y_star.mean()
    conf_intervals.loc[b, "upper"] = upper
 
for index, row in conf_intervals.iterrows():
    if row["lower"] <= true_mu and true_mu <= row["upper"]: 
        conf_intervals.loc[index, "lucky"] = True
    else:
        conf_intervals.loc[index, "lucky"] = False

lucky = conf_intervals.loc[conf_intervals.loc[:,"lucky"]==True]
unlucky = conf_intervals.loc[conf_intervals.loc[:, "lucky"]==False]

plt.scatter(lucky["b"], lucky["xbar"], color="b", label="lucky")
plt.scatter(unlucky["b"], unlucky["xbar"], color="r", label="unlucky")
plt.hlines(y=true_mu, xmin=-10, xmax=B+10, lw=1, color="r")
for index, row in lucky.iterrows():
    x, ymax, ymin = row["b"], row["upper"], row["lower"]
    plt.vlines(x=x, ymin=ymin, ymax=ymax, color="b", lw=1)
for index, row in unlucky.iterrows():
    x, ymax, ymin = row["b"], row["upper"], row["lower"]
    plt.vlines(x=x, ymin=ymin, ymax=ymax, color="r", lw=1)
    
plt.xlabel("b")
plt.ylabel("xbar")
plt.legend()
plt.show()

In [ ]:
# 6.10.2. 중심극한정리
np.random.seed(1708)
B = 10000
n = 10
xbars_star = np.zeros(B//n, dtype=float)
extra = np.random.randint(2, size=B)

fig = plt.figure(figsize=(15, 5))
fig.subplots_adjust(hspace=.5, wspace=.5)

for b in range(B//n):
    xbars_star[b] = np.mean(extra[b:b+10])
    
ax1 = fig.add_subplot(121)
ax1.set_title("Individual sleep time increase")
ax1.set_xlabel("c(0, 1)")
ax1.set_ylabel("Density")
sns.distplot(extra, ax=ax1, kde=False)

ax2 = fig.add_subplot(122)
ax2.set_title("Sample mean of 10 obs")
ax2.set_xlabel("xbars_star")
ax2.set_ylabel("Frequency")
sns.distplot(xbars_star, ax=ax2, kde=False)

plt.show()